In [8]:
import time
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict
import pyomo.environ as pyo

from gep_main import run_model, prep_data
from gep_problem import GEPProblem

import torch

DTYPE = torch.float64

torch.set_default_dtype(DTYPE)

# DEVICE = (
#     torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# )

DEVICE = "cpu"

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

HIGHS  = "HiGHS"
GUROBI = "Gurobi"

SAMPLE_DURATION = 12 # 12 hours

## Step 0: Activate environment - ensure consistency accross computers
# print("Reading the data")
# print("Activating the environment")


In [10]:

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

print("Initializing the solver")
optimizer_name = data["optimizer_config"]["solver"]

# Determine the optimizer
if optimizer_name == HIGHS:
    raise NotImplementedError(f"{optimizer_name}: Not implemented")
elif optimizer_name == GUROBI:
    
    print(f"Using {GUROBI}")
    optimizer = "gurobi_direct"
else:
    raise ValueError(f"{optimizer_name}: Not implemented")

for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap = prep_data(experiment_instance)

    T_ranges = [range(i, i + SAMPLE_DURATION, 1) for i in range(1, len(T), SAMPLE_DURATION)]
    objective_values = []
    times = []
    for t in T_ranges[:1]:
        # Run one experiment for j repeats
        GUROBI_MODEL, solver, time_taken = run_model(experiment_instance, t, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap)
        
        # print("Print values for all variables")
        # for v in model.component_data_objects(pyo.Var):
        #     print(str(v), v.value)

        objective_values.append(GUROBI_MODEL.obj())
        times.append(time_taken)

    # Write the number to the file
    # file_path = os.path.join("outputs/Gurobi", f"samplesize_{str(SAMPLE_DURATION)}_node_{len(N)}_gen_{len(G)}_lines_{len(L)}.txt")
    # with open(file_path, "w") as file:
    #     file.write(f"N: {str(N)}\n")
    #     file.write(f"G: {str(G)}\n")
    #     file.write(f"L: {str(L)}\n")
    #     file.write(f"Obj value mean: {str(np.mean(objective_values))}\n")
    #     file.write(f"Time taken mean: {str(np.mean(times))}\n")

Parsing the config file
Initializing the solver
Using Gurobi
Wrangling the input data
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter Method to value 2
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
Method  2
Crossover  0

Optimize a model with 140 rows, 113 columns and 467 nonzeros
Model fingerprint: 0x93c00021
Coefficient statistics:
  Matrix range     [7e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 3e+04]
  RHS range        [4e+03, 3e+04]
Presolve removed 126 rows and 104 columns
Presolve time: 0.00s
Presolved: 14 rows, 9 columns, 42 non

In [3]:
import pickle
import time
from gep_config_parser import *
from data_wrangling import dataframe_to_dict

from primal_dual import PrimalDualTrainer
from gep_problem import GEPProblem
import sys
import torch

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

def run_model(inputs, args):
    
    if outputs_config["terminal"]["input_plots"]:
        pass

    print("Wrangling the input data")

    # Extract sets
    T = inputs["times"] # [1, 2, 3, ... 8760] ---> 8760
    G = inputs["generators"] # [('Country1', 'EnergySource1'), ...] ---> 107
    L = inputs["transmission_lines"] # [('Country1', 'Country2'), ...] ---> 44
    N = inputs["nodes"] # ['Country1', 'Country2', ...] ---> 20

    ### SET UP CUSTOM CONFIG ###
    # N = ['BEL', 'FRA', 'GER', 'NED'] # 4 nodes
    N = ['BEL', 'GER', 'NED'] # 3 nodes
    # G = [('BEL', 'SunPV'), ('FRA', 'SunPV'), ('GER', 'SunPV'), ('NED', 'SunPV')] # 4 generators
    G = [('BEL', 'SunPV'), ('GER', 'SunPV'), ('NED', 'SunPV')] # 3 generators
    # L = [('BEL', 'FRA'), ('BEL', 'GER'), ('BEL', 'NED'), ('GER', 'FRA'), ('GER', 'NED')] # 5 lines
    L = [('BEL', 'GER'), ('BEL', 'NED'), ('GER', 'NED')] # 3 lines

    # Extract time series data
    pDemand = dataframe_to_dict(
        inputs["demand_data"],
        keys=["Country", "Time"],
        value="Demand_MW"
    )
    pGenAva = dataframe_to_dict(
        inputs["generation_availability_data"],
        keys=["Country", "Technology", "Time"],
        value="Availability_pu"
    )

    # Extract scalar parameters
    pVOLL = inputs["value_of_lost_load"]

    # WOP
    # Scale inversely proportional to times (T)
    pWeight = inputs["representative_period_weight"]

    pRamping = inputs["ramping_value"]

    # Extract generator parameters
    pInvCost = dataframe_to_dict(
        inputs["generation_data"],
        keys=["Country", "Technology"],
        value="InvCost_kEUR_MW_year"
    )
    pVarCost = dataframe_to_dict(
        inputs["generation_data"],
        keys=["Country", "Technology"],
        value="VarCost_kEUR_per_MWh"
    )
    pUnitCap = dataframe_to_dict(
        inputs["generation_data"],
        keys=["Country", "Technology"],
        value="UnitCap_MW"
    )

    # Extract line parameters
    pExpCap = dataframe_to_dict(
        inputs["transmission_lines_data"],
        keys=["CountryA", "CountryB"],
        value="ExpCap_MW"
    )
    pImpCap = dataframe_to_dict(
        inputs["transmission_lines_data"],
        keys=["CountryA", "CountryB"],
        value="ImpCap_MW"
    )

    # We need to sort the dictionaries for changing to tensors!
    pDemand = dict(sorted(pDemand.items()))
    pGenAva = dict(sorted(pGenAva.items()))
    pInvCost = dict(sorted(pInvCost.items()))
    pVarCost = dict(sorted(pVarCost.items()))
    pUnitCap = dict(sorted(pUnitCap.items()))
    pExpCap = dict(sorted(pExpCap.items()))
    pImpCap = dict(sorted(pImpCap.items()))


    print("Creating problem instance")
    problem = GEPProblem(T, G, L, N, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap, sample_duration=12)
    run_name = "2024-12-09"
    save_dir = os.path.join('outputs', 'PDL',
        run_name + "-" + str(time.time()).replace('.', '-'))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
        pickle.dump(args, f)

    # Run PDL
    print("Training the PDL")
    trainer = PrimalDualTrainer(problem, args, save_dir)
    primal_net, dual_net, stats = trainer.train_PDL()

    return problem, primal_net, dual_net, stats



Running on cpu
Parsing the config file


In [ ]:
args = {
        # "K": 2,
        "K": 20,
        # "L": 10,
        "L": 250,
        "tau": 0.1,
        "rho": 10000000,
        # "rho": 1e-5,
        "rho_max": sys.maxsize * 2 + 1,
        "alpha": 1000,
        # "alpha": 2,
        # "batch_size": 584, # Full training set!
        "batch_size": 10000,
        "hidden_size": 500,
        # "hidden_size": 1000,
        "primal_lr": 1e-4,
        "dual_lr": 1e-4,
        # "primal_lr": 1e-5,
        # "dual_lr": 1e-5,
        # "decay": 0.99,
        "decay": 0.99,
        "patience": 10,
        "corrEps": 1e-4,
}

# Train the model:
for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    for j in range(experiment["repeats"]):
        # Run one experiment for j repeats
        problem, primal_net, dual_net, stats = run_model(experiment_instance, args)

Wrangling the input data
Creating problem instance
Size of train set: 584
Size of val set: 73
Size of mu: 285
Size of lambda: 36
Number of variables (size of y): 111
Number of inputs (size of X): 72
Training the PDL
Epoch 0 done. Time taken: 5.493730068206787. Rho: 10000000. Primal LR: 8.600583546412887e-05, Dual LR: 8.016305895390461e-05
0: p-loss: 7.6231E+17, obj. val 5.8698E+07, Max eq.: 2.4370E+04, Max ineq.: 1.0455E+05, Mean eq.: 9.8552E+03, Mean ineq.: 7.2903E+03

Epoch 1 done. Time taken: 5.368190765380859. Rho: 10000000. Primal LR: 6.825545950103872e-05, Dual LR: 6.361854860638712e-05
1: p-loss: 8.8263E+17, obj. val 5.1035E+07, Max eq.: 2.4386E+04, Max ineq.: 1.1304E+05, Mean eq.: 9.8015E+03, Mean ineq.: 7.8341E+03

Epoch 2 done. Time taken: 5.08319878578186. Rho: 10000000000.0. Primal LR: 5.639051904523878e-05, Dual LR: 5.099857462495656e-05
2: p-loss: 5.0342E+19, obj. val 6.8185E+07, Max eq.: 2.4758E+04, Max ineq.: 3.0893E+04, Mean eq.: 9.8580E+03, Mean ineq.: 1.3113E+03

Epo

In [14]:
def get_variable_values_as_list(var):
    # Check if the variable is indexed
    if var.is_indexed():
        return [var[idx].value for idx in var]
    else:
        # For scalar variables
        return [var.value]

In [20]:
# Y_tensor_path = "outputs/Gurobi/Y_FIRST_SAMPLE_samplesize_12_node_3_gen_3_lines_3"
# Y_Gurobi = torch.load(Y_tensor_path)
# Y_Gurobi = model
data = GEPProblem(T, G, L, N, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap, sample_duration=12)
problem = GEPProblem(T, G, L, N, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap, sample_duration=12)
Y_Gurobi = torch.tensor([[*get_variable_values_as_list(GUROBI_MODEL.vGenProd), \
*get_variable_values_as_list(GUROBI_MODEL.vLineFlow),
*get_variable_values_as_list(GUROBI_MODEL.vLossLoad),
*get_variable_values_as_list(GUROBI_MODEL.vGenInv)]], device=DEVICE)
# print(Y_Gurobi.shape)

X = problem.trainX[:1]
Y_out = primal_net(X).detach()
print(Y_out.shape)

# print(Y_Gurobi)
# print(Y_out)

# print(Y_Gurobi - Y_out)
print(GUROBI_MODEL.obj())
print(problem.obj_fn(Y_Gurobi).item())
print(problem.obj_fn(Y_out).item())
print(data.obj_fn(Y_Gurobi).item())
print(data.obj_fn(Y_out).item())

Size of train set: 584
Size of val set: 73
Size of mu: 285
Size of lambda: 36
Number of variables (size of y): 111
Number of inputs (size of X): 72
Size of train set: 584
Size of val set: 73
Size of mu: 285
Size of lambda: 36
Number of variables (size of y): 111
Number of inputs (size of X): 72
torch.Size([1, 111])
818648032.282804
818648032.282804
1013178902.5024923
818648032.282804
1013178902.5024923
